In [1]:
# necessary imports
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch import optim
from tqdm import tqdm
from dataload import MyDataset
# import all the models
from lgg_model import *

In [2]:
option = input("请选择是否储存单词模型（y/n）：")
if option == 'y' or option == 'Y':
    save_model = True
else:
    save_model = False

In [3]:
# generate dataset and dataloader
BS = 32
input_word_count = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_path = input("请输入数据集名称（无需添加后缀）：")
data_path = "texts/" + data_path + ".txt"
full_dataset = MyDataset(data_path, input_word_count, save_word_model=save_model) # set the save option to True for the first time
vocabulary_length = full_dataset.vocabulary_length

train_in_all = 0.8 # train_in_all is the proportion of the dataset that will be used for training

train_size = int(train_in_all * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])
train_dataloader = DataLoader(train_dataset, batch_size=BS, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BS, shuffle=True)

In [4]:
vocabulary_length

1944

In [5]:
from datetime import datetime as dt

now = str(dt.now())
time_path = now[:10] + "_" + now[11:13] + "_" + now[14:16] + "_" + now[17:19]
print(time_path)
tb_path = "hole/" + time_path
tb_train_path = tb_path + "/train"
tb_val_path = tb_path + "/val"
tb_train_path, tb_val_path

2022-04-01_10_12_06


('hole/2022-04-01_10_12_06/train', 'hole/2022-04-01_10_12_06/val')

In [6]:
from torch.utils.tensorboard import SummaryWriter 
writer_train = SummaryWriter(tb_train_path)
writer_val = SummaryWriter(tb_val_path)

In [7]:
# some components in the training process
LR = 0.001 # the learning rate of 0.001 is still too large, maybe needs lr_decay or batch_norm
num_epoches = 20
net = vanilla_GRU(vocabulary_length, 50, 100, 2).to(device)
optimizer = optim.Adam(net.parameters(), lr=LR) 
criterion = nn.CrossEntropyLoss()

# 2022/2/27 add a lr decay controller
lr_decay_rate = 0.99
ctrl = torch.optim.lr_scheduler.ExponentialLR(optimizer, lr_decay_rate)

In [8]:
# print the hyperparameters
print("The hyperparameters are as follows:")
print("The dataset is:", data_path)
print("The learning rate is:", LR)
print("The number of epochs is:", num_epoches)
print("The batch size is:", BS)
print("The input word count is:", input_word_count)
print("The vocabulary length is:", vocabulary_length)
print("The ratio of train/val is:", int(train_in_all/(1-train_in_all)), ":", 1)
print("The network is:", net)
print("The device is:", device)
print("The optimizer is:", optimizer)
print("The criterion is:", criterion)
print("The lr decay schedule is:", ctrl, ", and the lr decay rate is:", lr_decay_rate)

The hyperparameters are as follows:
The dataset is: texts/hole02.txt
The learning rate is: 0.001
The number of epochs is: 20
The batch size is: 32
The input word count is: 10
The vocabulary length is: 1944
The ratio of train/val is: 4 : 1
The network is: vanilla_GRU(
  (Embedding): Embedding(1944, 50)
  (GRU): GRU(50, 100, num_layers=2, batch_first=True, dropout=0.5)
  (Linear): Linear(in_features=100, out_features=1944, bias=True)
)
The device is: cpu
The optimizer is: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 0.001
    lr: 0.001
    maximize: False
    weight_decay: 0
)
The criterion is: CrossEntropyLoss()
The lr decay schedule is: <torch.optim.lr_scheduler.ExponentialLR object at 0x7ff2f4439dc0> , and the lr decay rate is: 0.99


In [9]:
# start training!
for epoch in tqdm(range(num_epoches)):
    # train
    net.train()
    train_loss = 0
    for i, data in enumerate(train_dataloader):
        data = data.to(device)
        data = data.to(torch.long)
        label = data[:,1:]
        out = net(data)[:,:-1,:]
        out = torch.transpose(out, 2, 1)

        optimizer.zero_grad()
        loss = criterion(out, label)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    train_avg_loss = train_loss / len(train_dataloader)
    writer_train.add_scalar('Loss/Epoch', train_avg_loss, epoch+1) # epoch+1 because epoch starts from 0
    writer_train.flush()
    ctrl.step() # lr decay
    
    # validation
    net.eval()
    val_loss = 0
    with torch.no_grad():
        for i, data in enumerate(val_dataloader):
            data = data.to(device)
            data = data.to(torch.long)
            label = data[:,1:]
            out = net(data)[:,:-1,:]
            out = torch.transpose(out, 2, 1)
            loss = criterion(out, label)
            val_loss += loss.item()
    
    val_avg_loss = val_loss / len(val_dataloader)
    writer_val.add_scalar('Loss/Epoch', val_avg_loss, epoch+1) # epoch+1 because epoch starts from 0
    writer_val.flush()
    
print("Finish training!")

100%|██████████| 20/20 [40:07<00:00, 120.36s/it]

Finish training!


In [10]:
# if you want to save your language model...
model_name = input("请输入语言模型的名称：")
model_name = model_name + '_' + time_path
torch.save(net, "lgg_model_paths/" + model_name)

In [11]:
print("Now the learning rate is:", optimizer.param_groups[0]['lr'])

Now the learning rate is: 0.0008179069375972306
